<a href="https://colab.research.google.com/github/Minwoo-study/Korean_Classic_DB/blob/main/%ED%95%9C%EA%B5%AD%EA%B3%A0%EC%A0%84%EC%A2%85%ED%95%A9DB_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국고전종합DB 원문 및 번역문 전처리

In [ ]:
import pandas as pd
# import pickle5 as pickle
import pickle
import re
import regex
from tqdm import tqdm
tqdm.pandas()

from kiwipiepy import Kiwi, Option
kiwi = Kiwi()
kiwi.prepare()

from kss import split_sentences
import itertools
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib as mpl
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumBarunGothic')
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
import matplotlib.colors as clr


In [ ]:
with open('./한국문집총간_translate.pickle', "rb") as fh:
  data = pickle.load(fh)

In [ ]:
#총 105624 개의 문단(페이지)
data

,ID,원문,번역문
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前..."
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)..."
...,...,...,...
105619,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)..."
105620,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋..."
105621,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것..."
105622,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格..."


In [ ]:
#번역문 크롤링 실패한 목록
data[data['번역문'] =='번역문 오류']

,ID,원문,번역문
2976,ITKC_MO_0011A_0170_010_0020,원문 오류,번역문 오류
12002,ITKC_MO_0049A_0010_010_0040,원문 오류,번역문 오류
12003,ITKC_MO_0049A_0010_010_0050,원문 오류,번역문 오류
27214,ITKC_MO_0189A_0160_010_0010,원문 오류,번역문 오류
27215,ITKC_MO_0189A_0160_010_0020,원문 오류,번역문 오류
...,...,...,...
37540,ITKC_MO_0273A_0360_010_0520,원문 오류,번역문 오류
37541,ITKC_MO_0273A_0360_010_0530,원문 오류,번역문 오류
46200,ITKC_MO_0299A_0130_010_0010,원문 오류,번역문 오류
47243,ITKC_MO_0311A_0070_010_0490,원문 오류,번역문 오류


In [ ]:
#번역문 없는 것 지우기 => 105395개의 문단
data = data[data['번역문']!='번역문 오류']
data

,ID,원문,번역문
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前..."
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)..."
...,...,...,...
105619,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)..."
105620,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋..."
105621,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것..."
105622,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格..."


In [ ]:
data[data['원문']== '원문 오류']

,ID,원문,번역문
2526,ITKC_MO_0005A_0040_020_0120,원문 오류,"\n\n칠언고시인데, 여기에는 수록하지 않는다.\n\n"
8764,ITKC_MO_0024A_0160_050_0010,원문 오류,\n백암산 정토사기(白巖山淨土寺記) 없어졌다.\n\n【안】 이색(李穡)의 정토사(淨...
8765,ITKC_MO_0024A_0160_060_0010,원문 오류,\n본조의 칙위성지의 발어 정축 [本朝勅慰聖旨跋語 丁丑] 없어졌다.\n\n
8766,ITKC_MO_0024A_0160_070_0010,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8767,ITKC_MO_0024A_0160_080_0010,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8768,ITKC_MO_0024A_0160_080_0020,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8769,ITKC_MO_0024A_0160_080_0030,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8770,ITKC_MO_0024A_0160_080_0040,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8771,ITKC_MO_0024A_0160_080_0050,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8772,ITKC_MO_0024A_0160_080_0060,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...


### 겹치는 부분 제거

In [ ]:
# id 겹치는 것 = 없음
data[data.duplicated(['ID'])==True]

,ID,원문,번역문


In [ ]:
#원문 겹치는 것 
data[data.duplicated(['원문'])==True]

,ID,원문,번역문
1615,ITKC_MO_0004A_0340_010_0050,上延昌侯,"\n연창후에게천경(天經 해[日]를 말한다)이 좌로 돌고 삭통(朔統)이 우로 도니, ..."
1683,ITKC_MO_0004A_0360_010_0040,敎書,"\n\n관고\n운운. 대개 지인(至人)의 경지는 사는 것을 즐거움으로 삼지 않고, ..."
1687,ITKC_MO_0004A_0360_010_0080,敎書,"\n\n교서\n운운. 대개 국가가 석문(釋門)의 품계를 만들어 놓은 것은, 그 본의..."
1689,ITKC_MO_0004A_0360_010_0100,敎書,\n\n교서\n운운. 대개 대부가 70세가 되면 법에 의하여 사퇴를 원하니 이것이 ...
1694,ITKC_MO_0004A_0360_010_0150,\n敎書,\n운운. 우리나라에서 척리(戚里)에 대하여 공(公)ㆍ후(侯)ㆍ백(伯) 3등의 작위...
...,...,...,...
102429,ITKC_MO_0650A_0210_010_0350,大老西歸殷室傾。天心不與獨夫橫。採薇叩馬齊東語。誤了淸風百世名。,\n\n대로 서쪽으로 돌아가자 은나라 기울어 / 大老西歸殷室傾천심은 독부의 횡포와 ...
102430,ITKC_MO_0650A_0210_010_0360,白沙浩漫漫。悲風日暮起。借問此何地。云是古易水。昔日燕儲君。送別入秦士。入秦夫如何。一死酬知...,\n흰모래 벌 넓고 아득한데 / 白沙浩漫漫슬픈 바람이 석양에 일어나네 / 悲風日暮起...
103574,ITKC_MO_0663A_0020_010_0390,朝,\n아침〔朝〕꿈에서 깨자 서창에 달 지려는데 / 夢覺西窓月欲低숲에 내리던 비 비로소...
104334,ITKC_MO_0663A_0170_010_0140,\n\n\n\n\n\n삽화 새창열기\n\n,\n\n\n\n\n\n\n삽화 새창열기\n\n\n


In [ ]:
#번역문 겹치는 것  => 논의 필요 
data[data.duplicated(['번역문'])==True]

,ID,원문,번역문
2250,ITKC_MO_0004A_0500_010_0320,江天暮雪,\n\n저물녘에 내리는 강천의 눈\n해 저물어 어둡지 않은 곳이 없는데 / 晩來無處...
2789,ITKC_MO_0011A_0060_010_0190,萬尋澄澈靑銅鏡。千尺逶迤白玉虹。怪底古今流不盡。層巖直上是龍宮。,\n\n평사(平沙)에 내리는 기러기\n줄줄이 점점이 가지런했다 비꼈다 / 行行點點整...
2935,ITKC_MO_0011A_0130_030_0020,問。孟子曰。夏后氏五十而貢。殷人七十而助。周人百畝而徹。其實皆什一也。又曰。仁政必自經界始。...,\n묻노라.《논어(論語)》를 읽을 때에는 언제나 여러 제자들이 물은 것을 자신이 직...
2936,ITKC_MO_0011A_0130_030_0030,問。帝王之統。若四時之相代。有不可紊。天命人心之所歸。又不可誣也。三五已前。遐哉邈乎。雖有載...,\n묻노라.《논어(論語)》를 읽을 때에는 언제나 여러 제자들이 물은 것을 자신이 직...
2937,ITKC_MO_0011A_0130_030_0040,問。論語曰。旣庶矣。富之。旣富矣。敎之。又曰。善人敎民七年。可使卽戎。又曰。善人爲邦百年。亦...,\n묻노라.《논어(論語)》를 읽을 때에는 언제나 여러 제자들이 물은 것을 자신이 직...
...,...,...,...
103579,ITKC_MO_0663A_0020_010_0440,擘岸揷靑冥。參天抱雲木。寺樓刱何年。巍然當其谷。傑閣始隱映。長廊乍起伏。燈炯照僧衣。鍾動隨佛...,\n출발을 기록하다〔紀發行〕약관의 나이에 산수 구경 좋아하니 / 弱歲喜幽賞진토는 편...
103580,ITKC_MO_0663A_0020_010_0450,仲秋深谷寒。行子心悄悄。一路稍依微。千山遞遶圍。苦竹不見外。村落時縹緲。其人虎爲羣。其屋日不...,\n출발을 기록하다〔紀發行〕약관의 나이에 산수 구경 좋아하니 / 弱歲喜幽賞진토는 편...
103581,ITKC_MO_0663A_0020_010_0460,\n乾坤闢鴻濛。山嶽氣所凝。茲峰一卷撮。萬里乃相仍。我來値秋半。風高天宇澄。穹林拂蒙鬱。絶壑...,\n출발을 기록하다〔紀發行〕약관의 나이에 산수 구경 좋아하니 / 弱歲喜幽賞진토는 편...
103582,ITKC_MO_0663A_0020_010_0470,昔遊大源寺。愛此無塵雜。吟哦共詩流。選搜携殘衲。徘徊盡淸奇。夢寐思騰踏。今秋興何長。雙屐無餘...,\n출발을 기록하다〔紀發行〕약관의 나이에 산수 구경 좋아하니 / 弱歲喜幽賞진토는 편...


In [ ]:
data = data.drop_duplicates(['번역문'])
data

,ID,원문,번역문
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前..."
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)..."
...,...,...,...
105619,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)..."
105620,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋..."
105621,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것..."
105622,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格..."


In [ ]:
# data_dup.to_pickle('한국문집총간_trans_duplicate.pickle')

### 번역문 한자 및 특수문자 제거

In [ ]:
# 전처리 함수화
# def preprocess(title,text):
#     text = re.sub(f'\n{title}\n\n<sep>','',text)
#     text = re.sub(f'^\n{title}\n\n','',text)
#     text = re.sub('<sep>\n\n$','',text)
#     text = re.sub('\n',' ',text)
#     text = re.sub('\=+ \d{1,} \=+(\<sep\>)*','',text)
    
#     #한자제거 추가
#     text = re.sub(r'\([\W]*\)', '', text)
#     text = re.sub('[一-龥]', '', text)
#     text = re.sub("[()]","", text)
    
#     text = regex.sub('\=+ \p{Hangul}+ \=+(\<sep\>)*','',text)
#     text = regex.sub('\=+ \p{Han}+ \=+(\<sep\>)*','',text)
#     text = text.strip()
#     text = re.sub('<sep>\n*$','',text)
#     text_list = text.split('<sep>')
#     text_list = [x.strip() for x in text_list if not re.match('\=+.+\=+',x)]
#     # text_list = [x for x in text_list if not re.match('\=+.+\=+',x)]
#     return text_list

In [ ]:
def preprocess(text):
    text = re.sub('\n',' ',text).strip()
    text = re.sub("[()《》〔〕]","", text)
    text = re.sub('[一-龥]', '', text)
    text = regex.sub('\(\p{Han}+\)','',text)
    return text

In [ ]:
preprocess(data['번역문'][0])

'예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리에 놓는 것은 물이 모든 맛의 근본임을 중시하려고 해서이다. 각종 화려한 무늬의 옷감이 아름답기는 하지만 제사 때에 거친 삼베로 동이를 덮는 것은 부녀자들이 하는 길쌈질의 시초를 돌아보고 귀하게 여기려 해서이다.     ”라고 하였다. 옛날의 군자가 그 근본과 시초를 반드시 중하게 여기려 한 것이 이와 같았다.우리 동방에 문장이 나와서 글을 지어 후세에 전할 수 있게 된 것은 고운 최공으로부터 비롯되었다. 그리고 우리 동방의 선비로서 북쪽으로 중국에 유학하여 문장으로 천하에 명성을 떨친 것도 최공으로부터 시작되었다. 최공의 글 가운데 후세에 전하는 것으로는 오직 계원필경과 중산복궤집 2부가 있을 뿐이다. 그러고 보면 이 2부의 서책이 또한 우리 동방 문장의 근본이요 시초라고 할 것이다.글을 숭상하는 우리 동방의 풍조가 아조에 이르러서는 더욱 빛나고 무르익어 집집마다 연허와 조유가 배출되면서 시와 문으로 문집을 이룬 것이 집을 가득 채울 정도로 엄청나게 많았다. 하지만 최공의 글을 아는 사람은 드물기만 하였다.내가 일찍이 근대 사람이 편찬한 동국서목을 보니 중산복궤집이 실려 있기에 널리 구해 보았지만 끝내 얻지 못하였다. 다만 계원필경 20권은 우리 집안에서 대대로 소장해 왔으므로 내가 어려서부터 진귀한 글로 알고서 음미해 왔다. 그러나 간혹 사람들과 이야기를 나누다 보면, 비록 박아하고 글을 잘하며 옛것을 좋아하는 자라고 할지라도 모두 그 글을 본 적이 없다고 말하고 있으니, 그러고 보면 이 글이 거의 없어질 지경에 이르렀다고 해야 할 것이다. 만약 이 글이 동방에 전하지 않게 된다면, 이는 현주를 태실에 진설하지 않는 것과 같고, 거친 삼베로 제사용 술동이를 덮지 않는 것과 같게 될 것이니, 어떻게 백성에게 근본을 잊지 말라고 가르칠 수가 있겠는가.세상에서는 간혹 공의 글이 모두 변려 사륙문으로서 옛 작자의 글과는 완전히 다르다고 비평하기도 한다. 그러나 공이 중국에 들어가서

In [ ]:
data=data.reset_index(drop=True)

In [ ]:
data['text_list'] = data.progress_apply(lambda x:preprocess(x['번역문']),axis=1)
data['text_list']

100%|██████████| 104127/104127 [00:05<00:00, 17671.18it/s]
/tmp/ipykernel_649961/1663847976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text_list'] = data.progress_apply(lambda x:preprocess(x['번역문']),axis=1)


0         예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...
1         계원필경집 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나 사적...
2         회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시어...
3         신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살펴보...
4         신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 돌아...
                                ...                        
105619    공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 휘준...
105620    부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 휘 ...
105621    이것은 우리 향산공의 수고로, 인출편이라고 하는 것이다. 공이 순국하신 지 6년이 ...
105622    글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...
105623    오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...
Name: text_list, Length: 104127, dtype: object

In [ ]:
# data_dup['text_list'] = data_dup['text_list'].astype(str)

In [ ]:
data['text_list']

0         예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...
1         계원필경집 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나 사적...
2         회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시어...
3         신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살펴보...
4         신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 돌아...
                                ...                        
105619    공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 휘준...
105620    부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 휘 ...
105621    이것은 우리 향산공의 수고로, 인출편이라고 하는 것이다. 공이 순국하신 지 6년이 ...
105622    글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...
105623    오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...
Name: text_list, Length: 104127, dtype: object

##### 문단 -> 문장

In [ ]:
# 문단 -> 문장 (오래걸림)
# data_dup['sentence'] = data_dup['text_list'].progress_map(lambda x:split_sentences(x, backend='mecab', disable_gc=True))
# data_dup['sentence']

In [ ]:
# 문단 -> 문장 (마침표 기준으로 분할)
data['sentence'] = data['text_list'].progress_map(lambda x:x.split('.'))
data['sentence']

100%|██████████| 104127/104127 [00:00<00:00, 199354.79it/s]
/tmp/ipykernel_649961/3797538806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentence'] = data['text_list'].progress_map(lambda x:x.split('.'))


0         [예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자...
1         [계원필경집 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나 사...
2         [회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시...
3         [신 모는 아룁니다, 금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...
4         [신 모는 아룁니다, 신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...
                                ...                        
105619    [공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다,  대사성 ...
105620    [부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다,  시조 ...
105621    [이것은 우리 향산공의 수고로, 인출편이라고 하는 것이다,  공이 순국하신 지 6년...
105622    [글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을...
105623    [오호라, 이 책은 우리 향산공의 유문이다,  공의 큰 절의가 이미 하늘에 있는 해...
Name: sentence, Length: 104127, dtype: object

In [ ]:
data = data.explode(['sentence']).reset_index(drop=True)

#### 형태소 분석


In [ ]:
## 토크나이징 함수 선언 (품사 포함)
def tokenize(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word + ('다' if tag.startswith('V') else '')+ '/'+ tag # 동사/형용사에는 '다'를 붙여줌 + / 품사
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S') and word not in stopwords] # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거

In [ ]:
kiwi.add_user_word('해동', 'NNG', 100000)

False

In [ ]:
kiwi.analyze('해동에서는')

[([Token(form='해동', tag='NNP', start=0, len=2),
   Token(form='에서', tag='JKB', start=2, len=2),
   Token(form='는', tag='JX', start=4, len=1)],
  -24.13312530517578)]

In [ ]:
token_list = []
stopwords = ['이다','하다','있다','위하']
for i,result in enumerate(kiwi.analyze(data['sentence'].tolist())):
      if i % 200000 ==0:
            print(i)
      try:
            res, score = result[0]
            token_list.append([word + ('다' if tag.startswith('V') else '')+ '/'+ tag # 동사/형용사에는 '다'를 붙여줌 + / 품사
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S') and word not in stopwords]) # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거)
      except UnicodeDecodeError:
            print('UnicodeDecodeError')
            token_list.append([])

0
200000
400000
600000
UnicodeDecodeError
800000
1000000
1200000
1400000


In [ ]:
data['token'] = pd.Series(token_list)
data['token']

0          [예기/NNG, 이르다/VV, 달다/VA, 술/NNG, 맛/NNG, 좋다/VA, 하...
1          [각종/NNG, 화려/XR, 하/XSA, 무늬/NNG, 옷감/NNG, 아름답다/VA...
2                                                    [하다/VV]
3          [옛날/NNG, 군자/NNG, 그/MM, 근본/NNG, 시초/NNG, 반드시/MAG...
4          [우리/NP, 동방/NNG, 문장/NNG, 나오다/VV, 글/NNG, 짓다/VV, ...
                                 ...                        
1584494    [만년/NNG, 이르다/VV, 옛/MM, 경전/NNG, 안다/VV, 깊다/VA, 산...
1584495    [경술년/NNG, 변고/NNG, 지금/NNG, 여/XSN, 년/NNB, 동안/NNG...
1584496    [다만/MAG, 유집/NNP, 어루만지다/VV, 길다/VA, 탄식/NNG, 하/XS...
1584497          [족/NNB, 질/XSN, 중/NNB, 균/NNG, 삼가/NNG, 쓰다/VV]
1584498                                                   []
Name: token, Length: 1584499, dtype: object

In [ ]:
#토큰 4개 이하인 것 확인
data[data['token'].map(lambda x:len(x)<=4)]

,ID,원문,번역문,text_list,sentence,token
2,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,”라고 하였다,[하다/VV]
28,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,”라고 하였다,[하다/VV]
35,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,,[]
70,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,계원필경집 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나 사적...,,[]
76,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시어...",”라고 하였습니다,[하다/VV]
...,...,...,...,...,...,...
1584483,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...",”라고 하는 것이다,"[하다/VV, 것/NNB, 이다/VCP]"
1584484,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...",족손 강호가 적는다,"[족/NNB, 손/NNG, 강호/NNP, 적다/VA]"
1584485,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...",,[]
1584489,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...","”라고 하였으니, 참으로 그러하다","[하다/VV, 참으로/MAG, 그러/XR, 하/XSA]"


In [ ]:
# 불용어 제거
stopwords = ['하다/VV','하/XSV','이다/VCP','하/XSA','하다/VV','있다/VV','않다/VX','없다/VA','하다/VX','되다/VV','있다/VX','아니다/VCN']

In [ ]:
data['token'] = data['token'].progress_map(lambda x:[w for w in x if not w in stopwords])

100%|██████████| 1584499/1584499 [00:11<00:00, 132153.20it/s]


In [ ]:
# 상위 n개 단어 확인
cnt = Counter(list(itertools.chain(*data['token'].tolist())))
cnt.most_common(30) # 상위 N개

[('것/NNB', 754672),
 ('들/XSN', 274292),
 ('말/NNG', 270483),
 ('그/MM', 226145),
 ('수/NNB', 219178),
 ('사람/NNG', 201718),
 ('일/NNG', 164703),
 ('이/MM', 163453),
 ('되/XSV', 155712),
 ('못하다/VX', 144413),
 ('이/NP', 139344),
 ('때/NNG', 131641),
 ('한/MM', 128068),
 ('공/NNG', 127014),
 ('나/NP', 112356),
 ('같다/VA', 108238),
 ('마음/NNG', 106520),
 ('년/NNB', 104873),
 ('지다/VX', 104355),
 ('어찌/MAG', 100327),
 ('보다/VV', 97486),
 ('모두/MAG', 94735),
 ('히/NNG', 92737),
 ('알다/VV', 89360),
 ('뜻/NNG', 87490),
 ('주다/VX', 86557),
 ('또/MAG', 86167),
 ('생각/NNG', 80947),
 ('자/NNB', 80238),
 ('이르다/VV', 72754)]

In [ ]:
# 토큰 5개 이상만 활용
data = data[data['token'].map(lambda x:len(x)>=5)]

In [ ]:
data = data.reset_index(drop=True)
data

,ID,원문,번역문,text_list,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,"[예기/NNG, 이르다/VV, 달다/VA, 술/NNG, 맛/NNG, 좋다/VA, 제..."
1,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,각종 화려한 무늬의 옷감이 아름답기는 하지만 제사 때에 거친 삼베로 동이를 덮는 ...,"[각종/NNG, 화려/XR, 무늬/NNG, 옷감/NNG, 아름답다/VA, 제사/NN..."
2,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,옛날의 군자가 그 근본과 시초를 반드시 중하게 여기려 한 것이 이와 같았다,"[옛날/NNG, 군자/NNG, 그/MM, 근본/NNG, 시초/NNG, 반드시/MAG..."
3,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,우리 동방에 문장이 나와서 글을 지어 후세에 전할 수 있게 된 것은 고운 최공으로부...,"[우리/NP, 동방/NNG, 문장/NNG, 나오다/VV, 글/NNG, 짓다/VV, ..."
4,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,그리고 우리 동방의 선비로서 북쪽으로 중국에 유학하여 문장으로 천하에 명성을 떨친...,"[그리고/MAJ, 우리/NP, 동방/NNG, 선비/NNG, 북쪽/NNG, 중국/NN..."
...,...,...,...,...,...,...
1377359,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...","공은 처음부터 학문을 하는 사람으로 자처하지 않고 천성 그대로 행하였으므로, 사람...","[공/NNG, 처음/NNG, 학문/NNG, 사람/NNG, 자처/NNG, 천성/NNG..."
1377360,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...",만년에 이르러서는 옛 경전을 안고 깊은 산으로 들어가 독실하게 공부하면서 감히 단...,"[만년/NNG, 이르다/VV, 옛/MM, 경전/NNG, 안다/VV, 깊다/VA, 산..."
1377361,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...","경술년의 변고로부터 지금까지 20여 년 동안, 인심과 세도가 나날이 더러운 데로 ...","[경술년/NNG, 변고/NNG, 지금/NNG, 여/XSN, 년/NNB, 동안/NNG..."
1377362,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...",다만 유집을 어루만지며 길게 탄식할 뿐이다,"[다만/MAG, 유집/NNP, 어루만지다/VV, 길다/VA, 탄식/NNG, 뿐/NNB]"


In [ ]:
data.to_pickle('한국문집총간_번역문_token.pickle')')

###원문 전처리

In [ ]:
#https://drive.google.com/file/d/1dryd1D5YTBkIPP06RriE125j1AlFQg_Z/view?usp=sharing
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1dryd1D5YTBkIPP06RriE125j1AlFQg_Z" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1dryd1D5YTBkIPP06RriE125j1AlFQg_Z" -o df_test.pkl

In [ ]:
with open('df_test.pkl', "rb") as fh:
  df_test = pickle.load(fh)

In [ ]:
df_test 

In [ ]:
tqdm.pandas()

In [ ]:
# 문단 -> 문장 (오래걸림)
df_test['sentence'] = df_test['원문'].progress_map(lambda x:[x.split('。')])
df_test['sentence']

100%|██████████| 104122/104122 [00:01<00:00, 92594.22it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0         [[記有之, 曰酒醴之美, 而玄酒明水之尙, 貴五味之本也, 黼黻文繡之美, 而疏布之尙, ...
1         [[桂苑筆耕集二十卷, 新羅孤雲崔公在唐淮南幕府時公私應酬之作, 而東還之後, 手編表進于朝...
2         [[淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集...
3         [[臣某言, 今月某日, 得進奏院狀報, 奉十一日宣下改廣明元年爲中和元年者, 展義龜城, ...
4         [[臣某言, 臣得進奏院狀報, 入南蠻通和使劉光裕等廻, 雲南通和兼進獻國信金銀器物疋段香藥...
                                ...                        
105619    [[公諱晩燾字觀必姓李, 退溪文純公后, 大司成彙濬娶弼善姜橒女无育, 繼娶僉樞宋在憲女, ...
105620    [[府君諱晩燾字觀必, 號響山, 又曰直齋, 姓李氏, 始祖諱碩, 麗末以眞寶縣吏中司馬, ...
105621    [[此吾響山公手藳之名訒出編者也, 殉國之粤六年丙辰, 吾黨之士, 就其中節略爲十九卷原集,...
105622    [[文者氣之所形也, 氣必充積而後文亦有格力, 故觀於文而其人槩可想矣, 然氣不可力以致, ...
105623    [[嗚呼, 此吾響山公遺文也, 公大節旣昭然如日星之在天, 其不朽何待乎文哉, 獨後死者之論...
Name: sentence, Length: 104122, dtype: object

In [ ]:
# flatten list
df_test['sentence'] = df_test['sentence'].progress_map(lambda x:(list(itertools.chain(*x))))
df_test['sentence']

100%|██████████| 104122/104122 [00:00<00:00, 143978.59it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0         [記有之, 曰酒醴之美, 而玄酒明水之尙, 貴五味之本也, 黼黻文繡之美, 而疏布之尙, 反...
1         [桂苑筆耕集二十卷, 新羅孤雲崔公在唐淮南幕府時公私應酬之作, 而東還之後, 手編表進于朝者...
2         [淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二...
3         [臣某言, 今月某日, 得進奏院狀報, 奉十一日宣下改廣明元年爲中和元年者, 展義龜城, 易...
4         [臣某言, 臣得進奏院狀報, 入南蠻通和使劉光裕等廻, 雲南通和兼進獻國信金銀器物疋段香藥馬...
                                ...                        
105619    [公諱晩燾字觀必姓李, 退溪文純公后, 大司成彙濬娶弼善姜橒女无育, 繼娶僉樞宋在憲女, 憲...
105620    [府君諱晩燾字觀必, 號響山, 又曰直齋, 姓李氏, 始祖諱碩, 麗末以眞寶縣吏中司馬, 生...
105621    [此吾響山公手藳之名訒出編者也, 殉國之粤六年丙辰, 吾黨之士, 就其中節略爲十九卷原集, ...
105622    [文者氣之所形也, 氣必充積而後文亦有格力, 故觀於文而其人槩可想矣, 然氣不可力以致, 其...
105623    [嗚呼, 此吾響山公遺文也, 公大節旣昭然如日星之在天, 其不朽何待乎文哉, 獨後死者之論,...
Name: sentence, Length: 104122, dtype: object

In [ ]:
df_test

,ID,원문,번역문,sentence
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,"[記有之, 曰酒醴之美, 而玄酒明水之尙, 貴五味之本也, 黼黻文繡之美, 而疏布之尙, 反..."
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,"[桂苑筆耕集二十卷, 新羅孤雲崔公在唐淮南幕府時公私應酬之作, 而東還之後, 手編表進于朝者..."
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...",[淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二...
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,"[臣某言, 今月某日, 得進奏院狀報, 奉十一日宣下改廣明元年爲中和元年者, 展義龜城, 易..."
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","[臣某言, 臣得進奏院狀報, 入南蠻通和使劉光裕等廻, 雲南通和兼進獻國信金銀器物疋段香藥馬..."
...,...,...,...,...
105619,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","[公諱晩燾字觀必姓李, 退溪文純公后, 大司成彙濬娶弼善姜橒女无育, 繼娶僉樞宋在憲女, 憲..."
105620,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","[府君諱晩燾字觀必, 號響山, 又曰直齋, 姓李氏, 始祖諱碩, 麗末以眞寶縣吏中司馬, 生..."
105621,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","[此吾響山公手藳之名訒出編者也, 殉國之粤六年丙辰, 吾黨之士, 就其中節略爲十九卷原集, ..."
105622,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","[文者氣之所形也, 氣必充積而後文亦有格力, 故觀於文而其人槩可想矣, 然氣不可力以致, 其..."


In [ ]:
df_sent = df_test.explode('sentence')
df_sent

,ID,원문,번역문,sentence
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,記有之
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,曰酒醴之美
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,而玄酒明水之尙
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,貴五味之本也
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,黼黻文繡之美
...,...,...,...,...
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",不知問學之爲何名而義理之爲何物
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",如先生者更於何叫起來也
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",只得拊遺卷爲之太息而已
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",族姪中均謹跋


In [ ]:
df_sent.to_pickle('한국문집총간_원문_문장분리.pickle')

In [ ]:
df_sent['token'] = df_sent['sentence'].progress_map(lambda x:','.join(x).split(','))

100%|██████████| 3907515/3907515 [00:22<00:00, 170683.60it/s]


In [ ]:
df_sent

,ID,원문,번역문,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,記有之,"[記, 有, 之]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,曰酒醴之美,"[曰, 酒, 醴, 之, 美]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,而玄酒明水之尙,"[而, 玄, 酒, 明, 水, 之, 尙]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,貴五味之本也,"[貴, 五, 味, 之, 本, 也]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,黼黻文繡之美,"[黼, 黻, 文, 繡, 之, 美]"
...,...,...,...,...,...
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",不知問學之爲何名而義理之爲何物,"[不, 知, 問, 學, 之, 爲, 何, 名, 而, 義, 理, 之, 爲, 何, 物]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",如先生者更於何叫起來也,"[如, 先, 生, 者, 更, 於, 何, 叫, 起, 來, 也]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",只得拊遺卷爲之太息而已,"[只, 得, 拊, 遺, 卷, 爲, 之, 太, 息, 而, 已]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",族姪中均謹跋,"[族, 姪, 中, 均, 謹, 跋]"


In [ ]:
#1개 이하의 글자 제거
df_sent_1 = df_sent[df_sent['token'].map(lambda x:len(x)>=2)]
df_sent_1

,ID,원문,번역문,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,記有之,"[記, 有, 之]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,曰酒醴之美,"[曰, 酒, 醴, 之, 美]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,而玄酒明水之尙,"[而, 玄, 酒, 明, 水, 之, 尙]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,貴五味之本也,"[貴, 五, 味, 之, 本, 也]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,黼黻文繡之美,"[黼, 黻, 文, 繡, 之, 美]"
...,...,...,...,...,...
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",人心世道日趍於汙,"[人, 心, 世, 道, 日, 趍, 於, 汙]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",不知問學之爲何名而義理之爲何物,"[不, 知, 問, 學, 之, 爲, 何, 名, 而, 義, 理, 之, 爲, 何, 物]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",如先生者更於何叫起來也,"[如, 先, 生, 者, 更, 於, 何, 叫, 起, 來, 也]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",只得拊遺卷爲之太息而已,"[只, 得, 拊, 遺, 卷, 爲, 之, 太, 息, 而, 已]"


In [ ]:
df_sent_1.to_pickle('한국문집총간_원문_token.pickle')